<a href="https://colab.research.google.com/github/drfperez/nautica/blob/main/VincentyCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

# ============================================================
# VINCENTY INVERS AMB WIDGETS INTERACTIUS
# ============================================================

!pip install mpmath ipywidgets

from mpmath import mp
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------------------------------------------
# PRECISIÓ
# ------------------------------------------------------------
mp.dps = 9
# Pots triar decimals, de 9 a 50

# ------------------------------------------------------------
# 0) INPUT INTERACTIU
# ------------------------------------------------------------
# Widgets per a latitud i longitud amb valors per defecte
phi1_widget = widgets.FloatText(value=46.494953, description='φ1 (°):')
lam1_widget = widgets.FloatText(value=-1.792091, description='λ1 (°):')
phi2_widget = widgets.FloatText(value=15.8640, description='φ2 (°):')
lam2_widget = widgets.FloatText(value=-61.5860, description='λ2 (°):')

# Botó per executar càlcul
calc_button = widgets.Button(description="Calcular distància")

# Mostrar widgets
display(phi1_widget, lam1_widget, phi2_widget, lam2_widget, calc_button)

# ------------------------------------------------------------
# Funció de càlcul
# ------------------------------------------------------------
def vincenty_inverse(phi1_deg, lam1_deg, phi2_deg, lam2_deg):
    clear_output(wait=True)

    print("="*120)
    print("COORDENADES D'ENTRADA (graus)")
    print(f"φ1 = {phi1_deg}  λ1 = {lam1_deg}")
    print(f"φ2 = {phi2_deg}  λ2 = {lam2_deg}")
    print("="*120)

    # Constants WGS-84
    a = mp.mpf("6378137.0")
    f = mp.mpf("1") / mp.mpf("298.257223563")
    b = a * (1 - f)

    print("Constants WGS-84:")
    print("a =", a, " f =", f, " b =", b)
    print("="*120)

    # Conversió a radians
    deg2rad = mp.pi / mp.mpf(180)
    phi1 = mp.mpf(phi1_deg) * deg2rad
    phi2 = mp.mpf(phi2_deg) * deg2rad
    lam1 = mp.mpf(lam1_deg) * deg2rad
    lam2 = mp.mpf(lam2_deg) * deg2rad

    # Diferència de longitud
    L = lam2 - lam1

    # Latituds reduïdes
    U1 = mp.atan((1 - f) * mp.tan(phi1))
    U2 = mp.atan((1 - f) * mp.tan(phi2))
    sinU1, cosU1 = mp.sin(U1), mp.cos(U1)
    sinU2, cosU2 = mp.sin(U2), mp.cos(U2)

    # Iteració de Vincenty
    lam = L
    tol = mp.mpf("1e-30")
    max_iter = 50

    print("ITERACIÓ DE VINCENTY - DETALLADA")
    print("-"*220)
    print(f"{'i':>2} | {'λ (rad)':>25} | {'Δλ':>25} | {'sinσ':>15} | {'cosσ':>15} | {'σ':>15} | {'sinα':>15} | {'cos²α':>15} | {'cos²σm':>15} | {'C':>15}")
    print("-"*220)

    for i in range(1, max_iter + 1):
        lam_prev = lam
        sin_lam = mp.sin(lam)
        cos_lam = mp.cos(lam)
        sin_sigma = mp.sqrt((cosU2*sin_lam)**2 + (cosU1*sinU2 - sinU1*cosU2*cos_lam)**2)
        cos_sigma = sinU1*sinU2 + cosU1*cosU2*cos_lam
        sigma = mp.atan2(sin_sigma, cos_sigma)
        sin_alpha = cosU1*cosU2*sin_lam / sin_sigma
        cos2_alpha = 1 - sin_alpha**2
        if cos2_alpha != 0:
            cos2_sigma_m = cos_sigma - 2*sinU1*sinU2/cos2_alpha
        else:
            cos2_sigma_m = mp.mpf(0)
        C = (f/16)*cos2_alpha*(4 + f*(4 - 3*cos2_alpha))
        lam = L + (1 - C)*f*sin_alpha*(sigma + C*sin_sigma*(cos2_sigma_m + C*cos_sigma*(-1 + 2*cos2_sigma_m**2)))
        dlam = lam - lam_prev

        print(f"{i:>2} | {str(lam):>25} | {str(dlam):>25} | {str(sin_sigma):>15} | {str(cos_sigma):>15} | {str(sigma):>15} | {str(sin_alpha):>15} | {str(cos2_alpha):>15} | {str(cos2_sigma_m):>15} | {str(C):>15}")

        if mp.fabs(dlam) < tol:
            print("-"*220)
            print("Convergència assolida a iteració", i)
            break

    # Distància final
    u2 = cos2_alpha*(a**2 - b**2)/(b**2)
    A = 1 + (u2/16384)*(4096 + u2*(-768 + u2*(320 - 175*u2)))
    B = (u2/1024)*(256 + u2*(-128 + u2*(74 - 47*u2)))
    delta_sigma = B*sin_sigma*(cos2_sigma_m + (B/4)*(cos_sigma*(-1 + 2*cos2_sigma_m**2) - (B/6)*cos2_sigma_m*(-3 + 4*sin_sigma**2)*(-3 + 4*cos2_sigma_m**2)))
    s = b*A*(sigma - delta_sigma)

    # Càlcul de rumbs
    alpha1 = mp.atan2(cosU2 * sin_lam, cosU1 * sinU2 - sinU1 * cosU2 * cos_lam)
    alpha2 = mp.atan2(cosU1 * sin_lam, -sinU1 * cosU2 + cosU1 * sinU2 * cos_lam)

    def to_bearing(az_rad):
        deg = mp.degrees(az_rad)
        return mp.fmod(deg + 360, 360)

    rumb_inicial = to_bearing(alpha1)
    rumb_final_retorn = to_bearing(alpha2 + mp.pi)  # Rumb final de retorn (reverse bearing)
    rumb_final_arribada = mp.fmod(rumb_final_retorn + 180, 360)  # Rumb final d'arribada (+180 mod 360)

    print("="*120)
    print("RESULTATS FINALS")
    print("Distància (m) =", s)
    print("Distància (km) =", s/1000)
    print("Distància (NM) =", s/mp.mpf(1852))
    print("Rumb inicial (°) =", rumb_inicial)
   # print("Rumb de retorn inicial (°) =", rumb_inicial)  # El rumb de retorn inicial seria el rumb final de retorn +180 mod360, però com és simètric, ajusta segons necessitat
   # print("Rumb de retorn final (°) =", rumb_final_retorn)
    print("Rumb final d'arribada (°) =", rumb_final_arribada)  # Afegit: rumb final +180 mod360

# ------------------------------------------------------------
# Connectar botó amb funció
# ------------------------------------------------------------
def on_button_click(b):
    vincenty_inverse(phi1_widget.value, lam1_widget.value,
                     phi2_widget.value, lam2_widget.value)

calc_button.on_click(on_button_click)

COORDENADES D'ENTRADA (graus)
φ1 = 46.494953  λ1 = -1.792091
φ2 = 15.864  λ2 = -61.586
Constants WGS-84:
a = 6378137.0  f = 0.00335281067  b = 6356752.31
ITERACIÓ DE VINCENTY - DETALLADA
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 i |                   λ (rad) |                        Δλ |            sinσ |            cosσ |               σ |            sinα |           cos²α |          cos²σm |               C
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 1 |               -1.04589314 |             -0.0022925525 |     0.847265497 |     0.531169631 |      1.01081588 |     -0.67681289 |     0.541924312 |    -0.197115992 |  0.000455146384
 